## Segmented Eratosthenes sieve
For the 100th algorithm I chose a segmented [Eratosthenes sieve](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes#Implementation) for primes up to $10^9$ implemented in [Cython](http://cython.org/). And this time the goal is to provide an algorithm that is as fast as possible.

There are several reasons why I diverged from Python and the usual type of implementation. Among others, I like breaking rules and I like optimizations. But what’s most important…

In most of the articles I have tried to provide as short and nice implementation as possible. I was aware that the algorithm could have been improved, but I didn’t do it since I would have to add another 5 or 10 lines. And here is my reasoning.

---

I have seen many people talking about time complexity and witnessed a common [mis]understanding of how asymptotical notations work. I have also seen many who do understand very well and [unlike textbooks] their evidence supports mine.

In theory, analysis of  [Big-O notation](https://en.wikipedia.org/wiki/Big_O_notation) starts with a theoretical computational model beneath the algorithm to study its behaviour in relation to data. With respect to algorithms, $f(n) = O(g(n))$ states that for large enough data algorithm $f$ requires at most number of steps required by algorithm $g$. And $f$ can be divided by any positive constant to make the claim to be true.

If you are interested, check my [Amortized Algorithm Analysis](https://medium.com/@tomas.bouda/amortized-algorithm-analysis-bb6a227be324) to see a small example of complexity analysis.

In practice, however, Big-O notation works a different way. Given two algorithms solving the same problem, there are [a common] data that **O(n.log n)** algorithm will perform worse than **O(n²)** algorithm. And it is not unlikely that it is the case for most of your data.

I have already touched the topic in this series. It is [almost] impossible to write a complex algorithm that performs best under any scenario. Standard libraries are usually written to perform well in expected situations and to survive the unexpected.

If, for example, you need to sort data on regular basis and your array has few tens of items, built-in sort is not the best algorithm. You can do many times faster, but the total time is negligible and you don’t have to care anyways. That means to perform well.

If you need to sort millions of records on regular basis, calling built-in sort is likely the worst thing you can do. You better use a different data structure like binary search trees to avoid batch sorting at all.

Or maybe not. My own evidence shows that the best performing algorithm is usually the one you would expect least. You simply have to try and give it a chance.

---

In practice, complexity analysis always starts with a [profiler](https://en.wikipedia.org/wiki/Profiling_%28computer_programming%29).

![day100-segmented_eratisthemes sieve_1](resource/day100-segmented_eratisthemes sieve_1.png)

Write a referential implementation and let the profiler show you any bottlenecks. Think about improvements, improve and profile again. And again.

That’s how I implemented segmented Eratosthenes sieve. I identified critical functions, wrote several implementations for each one of them and checked the results.

The main idea behind segmented sieve is that modern CPU can’t address a single byte and not even an integer. CPU has several layers of cache and whenever you address anywhere in memory, it has to fetch the block and store it inside cache, first.

This takes a lot of time and Eratosthenes sieve addressing wide consecutive areas of memory is simply wasteful. Fortunately, the algorithm can be rewritten to sieve primes in local segments that fit inside CPU cache.

![day100-segmented_eratisthemes sieve_2](resource/day100-segmented_eratisthemes sieve_2.png)

The gain is enormous and my implementation is able find number of primes below $10^6$ in time about .6 ms. Number of primes below $10^9$ can be found in about .8 s. For a reference, my machine is MacBook Pro 2014, 2.2GHz i7.

If you are using [Anaconda](https://www.continuum.io/downloads) distribution, the notebook will work fine. To run the algorithm outside of notebook, follow these steps.

* make sure you have [Cython](http://cython.org/) installed
* create file `day100.pyx` and put the code inside; do not forget to remove `%%cython` directive
* create and run file `main.py` with the following code

```python
import pyximport; pyximport.install()
import day100

print(day100.eratosthenes(10 ** 9))
```

In [5]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## algorithm

In [2]:
%%cython

from libc.stdlib cimport malloc, free

DEF LIMIT = 1024 * 31
DEF PRIME = 1024 * 4
DEF SIEVE = 1024 * 32

cdef inline int imin(int a, int b) nogil:
    return a if a < b else b

cdef inline int memset(char *p, int n) nogil:
    cdef:
        short *q = <short *>p
        int i, j = 0

    for i in range((n + 1) >> 1):
        j += q[i]
        q[i] = 0x0100

    return j >> 8

cdef int naive_sieve(char *sieve, int *primes, int *offsets, int n) nogil:
    cdef int i, j

    memset(sieve, n)

    for i in range(3, n, 2):
        if sieve[i]:
            j = i * i
            while j < n:
                sieve[j] = 0
                j += i << 1

            primes[0] = i
            offsets[0] = j
            primes += 1
            offsets += 1

    primes[0] = 0
    offsets[0] = 0

    return memset(sieve, n)

cdef int segmented_sieve(char *sieve, int *primes, int *offsets, int k, int n) nogil:
    cdef int i

    while primes[0]:
        i = offsets[0] - k
        while i < n:
            sieve[i] = 0
            i += primes[0] << 1
        offsets[0] = i + k

        primes += 1
        offsets += 1

    return memset(sieve, n)

cpdef int eratosthenes(int n) nogil:
    cdef:
        char *sieve
        int *primes
        int *offsets
        int k, total

    if n > LIMIT * LIMIT:
        return -1

    sieve = <char *>malloc(SIEVE)
    primes = <int *>malloc(PRIME * sizeof(int))
    offsets = <int *>malloc(PRIME * sizeof(int))

    total = naive_sieve(sieve, primes, offsets, imin(n, LIMIT))

    memset(sieve, SIEVE)
    k = LIMIT
    n -= LIMIT

    while n > 0:
        total += segmented_sieve(sieve, primes, offsets, k, imin(n, SIEVE))
        k += SIEVE
        n -= SIEVE

    free(sieve)
    free(primes)
    free(offsets)

    return total

## run

In [3]:
for i in range(1, 10):
    print('primes below 10**%d: %d' % (i, eratosthenes(10**i)))

primes below 10**1: 4
primes below 10**2: 25
primes below 10**3: 168
primes below 10**4: 1229
primes below 10**5: 9592
primes below 10**6: 78498
primes below 10**7: 664579
primes below 10**8: 5761455
primes below 10**9: 50847534


## timeit

In [4]:
%timeit eratosthenes(1024 * 31)
%timeit eratosthenes(10**6)
%timeit eratosthenes(10**7)
%timeit eratosthenes(10**8)
%timeit eratosthenes(10**9)

49.2 µs ± 602 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
664 µs ± 7.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.01 ms ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
77.7 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
896 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
